In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to TF-IDF and return the dataset
def convert_to_tfidf(input_file, text_column):
    df = pd.read_excel(input_file)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df[text_column].astype(str))
    print(f"TF-IDF matrix dimensions for '{text_column}': {tfidf_matrix.shape}")
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return tfidf_df, df


# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred, ensuring they are 1D
    all_labels = sorted(list(set(y_test.tolist()) | set(test_pred.flatten().tolist())))  # Convert to list first
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred.flatten(), columns=all_labels).reindex(columns=all_labels, fill_value=0)  # Flatten test_pred
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")



def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cat Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
text_column = 'answer'

# Convert text column to TF-IDF and get the original DataFrame
tfidf_data, original_data = convert_to_tfidf(input_file, text_column)

# Prepare target variable with shifted labels
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_data, y, test_size=0.3, random_state=42)

# Run all models with hyperparameter tuning
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TF-IDF matrix dimensions for 'answer': (681, 1772)
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 9}
KNN Model Metrics:
Train Accuracy: 0.7395
Train Precision: 0.7249
Train Recall: 0.7395
Train F1 Score: 0.7093
Train Confusion Matrix:
[[  6  13  13]
 [  2  54  76]
 [  0  20 292]]

Test Accuracy: 0.7415
Test Precision: 0.7275
Test Recall: 0.7415
Test F1 Score: 0.7007
Test Confusion Matrix:
[[  1   3   9]
 [  0  11  30]
 [  0  11 140]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      73.95%
Test Accuracy       74.15%
Train-Test Gap      0.20%
ROC-AUC             0.60
Overfitting         No


Running Random Forest model...
Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9853
Train Precision: 0.9854
Tr

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.9874
Train Precision: 0.9875
Train Recall: 0.9874
Train F1 Score: 0.9873
Train Confusion Matrix:
[[ 32   0   0]
 [  0 127   5]
 [  0   1 311]]

Test Accuracy: 0.6878
Test Precision: 0.6707
Test Recall: 0.6878
Test F1 Score: 0.6784
Test Confusion Matrix:
[[  1   6   6]
 [  3  14  24]
 [  3  22 126]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      98.74%
Test Accuracy       68.78%
Train-Test Gap      29.96%
ROC-AUC             0.62
Overfitting         Yes


Running XGBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 0.8, 'n_estimators': 50, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9601
Train Precision: 0.9613
Train Recall: 0.9601
Train F1 Score: 0.9596
Train Confusion Matrix:
[[ 28   0   4]
 [  0 119  13]
 [  0   2 310]]

Test Accuracy: 0.7415
Test Precision: 0.7126
Test Recall: 0.7415
Test F1 Score: 0.7215
Test Confusion Matrix:
[[  2   4   7]
 [  2  15  24]
 [  1  15 135]]


Performance Table for XG BOOST:
Metric              Value     
------------------------------
Train Accuracy      96.01%
Test Accuracy       74.15%
Train-Test Gap      21.86%
ROC-AUC             0.65
Overfitting         Yes


Running AdaBoost model...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 10}
CatBoost Model Metrics:
Train Accuracy: 0.9895
Train Precision: 0.9895
Train Recall: 0.9895
Train F1 Score: 0.9895
Train Confusion Matrix:
[[ 32   0   0]
 [  0 128   4]
 [  0   1 311]]

Test Accuracy: 0.7756
Test Precision: 0.7572
Test Recall: 0.7756
Test F1 Score: 0.7544
Test Confusion Matrix:
[[  4   3   6]
 [  1  15  25]
 [  0  11 140]]


Performance Table for Cat Boost:
Metric              Value     
------------------------------
Train Accuracy      98.95%
Test Accuracy       77.56%
Train-Test Gap      21.39%
ROC-AUC             0.67
Overfitting         Yes


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9771
Train Recall: 0.9769
Train F1 Score: 0.9768
Train Confusion Matrix:
[[ 31   0   1]
 [  0 124   8]
 [  0   2 310]]

Test Accuracy: 0.7366
Test Precision: 0.6881
Test Recall: 0.7366
Test F1 Score: 0.7018
Test Confusion Matrix:
[[  1   5   7]
 [  2  11  28]
 [  0  12 139]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      97.69%
Test Accuracy       73.66%
Train-Test Gap      24.03%
ROC-AUC             0.62
Overfitting         Yes


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9853
Train Precision: 0.9853
Train Recall: 0.9853
Train F1 Score: 0.9852
Train Confusion Matrix:
[[ 32   0   0]
 [  0 127   5]
 [  0   2 310]]

Test Accuracy: 0.7024
Test Precision: 0.7091
Test Recall: 0.7024
Test F1 Score: 0.7055
Test Confusion Matrix:
[[  3   7   3]
 [  5  16  20]
 [  4  22 125]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      98.53%
Test Accuracy       70.24%
Train-Test Gap      28.29%
ROC-AUC             0.68
Overfitting         Yes




In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to TF-IDF and return the dataset
def convert_to_tfidf(input_file, text_column):
    df = pd.read_excel(input_file)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df[text_column].astype(str))
    print(f"TF-IDF matrix dimensions for '{text_column}': {tfidf_matrix.shape}")
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return tfidf_df, df


# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred, ensuring they are 1D
    all_labels = sorted(list(set(y_test.tolist()) | set(test_pred.flatten().tolist())))  # Convert to list first
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred.flatten(), columns=all_labels).reindex(columns=all_labels, fill_value=0)  # Flatten test_pred
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")



def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cat Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
text_column = 'answer'

# Convert text column to TF-IDF and get the original DataFrame
tfidf_data, original_data = convert_to_tfidf(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(tfidf_data, variance_threshold=0.95)

# Prepare target variable with shifted labels
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models with hyperparameter tuning
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TF-IDF matrix dimensions for 'answer': (681, 1772)
Number of components selected: 383
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 16}
KNN Model Metrics:
Train Accuracy: 0.6954
Train Precision: 0.6882
Train Recall: 0.6954
Train F1 Score: 0.6418
Train Confusion Matrix:
[[  3  10  19]
 [  0  34  98]
 [  0  18 294]]

Test Accuracy: 0.7366
Test Precision: 0.6534
Test Recall: 0.7366
Test F1 Score: 0.6835
Test Confusion Matrix:
[[  0   4   9]
 [  0   9  32]
 [  0   9 142]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      69.54%
Test Accuracy       73.66%
Train-Test Gap      4.12%
ROC-AUC             0.58
Overfitting         No


Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.9853
Train Precision: 0.9853
Train Recall: 0.9853
Train F1 Score: 0.9852
Train Confusion Matrix:
[[ 32   0   0]
 [  0 127   5]
 [  0   2 310]]

Test Accuracy: 0.6537
Test Precision: 0.6815
Test Recall: 0.6537
Test F1 Score: 0.6607
Test Confusion Matrix:
[[  1   6   6]
 [  1  21  19]
 [  3  36 112]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      98.53%
Test Accuracy       65.37%
Train-Test Gap      33.16%
ROC-AUC             0.63
Overfitting         Yes


Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9895
Train Precision: 0.9895
Train Recall: 0.9895
Train F1 Score: 0.9895
Train Confusion Matrix:
[[ 32   0   0]
 [  0 128   4]
 [  0   1 311]]

Test Accuracy: 0.7463
Test Precision: 0.6694
Test Recall: 0.7463
Test F

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 100, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.8319
Train Precision: 0.8273
Train Recall: 0.8319
Train F1 Score: 0.8281
Train Confusion Matrix:
[[ 32   0   0]
 [  0  83  49]
 [  0  31 281]]

Test Accuracy: 0.6293
Test Precision: 0.6168
Test Recall: 0.6293
Test F1 Score: 0.6176
Test Confusion Matrix:
[[  1   4   8]
 [  1   9  31]
 [  1  31 119]]


Performance Table for Ada Boost:
Metric              Value     
------------------------------
Train Accuracy      83.19%
Test Accuracy       62.93%
Train-Test Gap      20.27%
ROC-AUC             0.53
Overfitting         Yes


Running CatBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [2]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to TF-IDF and return the dataset
def convert_to_tfidf(input_file, text_column):
    df = pd.read_excel(input_file)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df[text_column].astype(str))
    print(f"TF-IDF matrix dimensions for '{text_column}': {tfidf_matrix.shape}")
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return tfidf_df, df


# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred, ensuring they are 1D
    all_labels = sorted(list(set(y_test.tolist()) | set(test_pred.flatten().tolist())))  # Convert to list first
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred.flatten(), columns=all_labels).reindex(columns=all_labels, fill_value=0)  # Flatten test_pred
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")



def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'
text_column = 'answer'

# Convert text column to TF-IDF and get the original DataFrame
tfidf_data, original_data = convert_to_tfidf(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(tfidf_data, variance_threshold=0.95)

# Prepare target variable with shifted labels
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models with hyperparameter tuning
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive
TF-IDF matrix dimensions for 'answer': (681, 1772)
Number of components selected: 383
Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.9643
Train Precision: 0.9648
Train Recall: 0.9643
Train F1 Score: 0.9639
Train Confusion Matrix:
[[ 31   0   1]
 [  0 119  13]
 [  0   3 309]]

Test Accuracy: 0.7073
Test Precision: 0.6822
Test Recall: 0.7073
Test F1 Score: 0.6889
Test Confusion Matrix:
[[  1   5   7]
 [  2  14  25]
 [  0  21 130]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      96.43%
Test Accuracy       70.73%
Train-Test Gap      25.70%
ROC-AUC             0.62
Overfitting         Yes


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9664
Train Precision: 0.9668
Train Recall: 0.9664
Train F1 Score: 0.9661
Train Confusion Matrix:
[[ 31   0   1]
 [  0 120  12]
 [  0   3 309]]

Test Accuracy: 0.6585
Test Precision: 0.6711
Test Recall: 0.6585
Test F1 Score: 0.6645
Test Confusion Matrix:
[[  3   4   6]
 [  5  14  22]
 [  6  27 118]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      96.64%
Test Accuracy       65.85%
Train-Test Gap      30.78%
ROC-AUC             0.62
Overfitting         Yes


